### 희소성 해결하기

기존의 n-gram 기반 언어 모델은 간편하지만 훈련 데이터에서 보지 못한 단어의 조합에는 꽤 취약한 부분이 있습니다. n-gram 기반의 언어 모델은 단어간 유사도를 알지 못하기 때문입니다. 예를 들어 우리에게 훈련 코퍼스로 아래와 같은 문장이 주어졌다고 가정합니다

<br></br>
* 고양이는 좋은 반려동물입니다.
<br></br>


사람은 각 단어 사이의 유사도를 알기때문에 다음중 어떤 확률이 클지 알수 있습니다.

<br></br>
* P(반려동물|강아지는, 좋은)
* P(반려동물|자동차는, 좋은)
<br></br>

하지만 기존에는 훈련 코퍼스에 해당 n-gram이 존재하지 않으면 출현 빈도를 계산할 수 없었기 때문에 확률을 구할 수 없고, 따라서 확률간 비교도 할 수 없었습니다. 

강아지가 개의 새끼고, 포오류에 속하는 가축에 해당한다는 지식이 없을지라도, 강아지와 고양이 사이의 유사도가 자동차와 고양이 사이의 유사도보다 높다는 것은 알기때문에 자동차보다는 강아지에 대한 반려동물의 확률이 너 높을을 유추할 수 있습니다. 하지만 n-gram방식의 언어모델은 단어간 유사도를 구할 수 없기때문에, 이처럼 훈련 코퍼스에서 보지 못한 단어의 조합인 n-gram에 대해 효과적으로 대처할 수 없습니다.

신경망 언어 모델, 즉 Neural Network Language Model (NNLM)은 단어 임베딩을 사용하여 단어를 차원 축소함으로써 강아지와 고양이를 비슷한 덴스 벡터로 학습하고, 자동차와 고양이보다 훨씬 높은 유사도를 가지게 됩니다. 따라서 NNLM이 훈련 코퍼스에서 보지 못했던 단어의 조합을 만나더라도, 비슷한 훈련 데이터로부터 배운 것과 유사하게 대처할 수 있습니다.

NNLM은 다양한 형태를 가질 수 있지만, 여기서는 가장 효율적이고 흔한 형태인 LSTM을 활용한 방식에 관해 살펴보겠습니다.

### RNNLM

RNN을 사용한 언어 모델인 RNNLM은 다음과 같은 구조를 지닙니다.

<br></br>
![](./images/9-5-2-rnnlm.jpg)
<br></br>

기존의 n-gram 언어 모델은 각각의 단어를 불연속적인 데이터로 취급했기 때문에, 단어 조합의 길이가 길어지면 희소성 문제로 어려움을 겪었습니다. 따라서 마르코프 가정을 통해 n-1 이전까지의 단어만 조건부로 사용하여 확률을 근사했습니다.

하지만 RNNLM은 단어 임베딩을 통해 덴스 벡터로 만들어 희소성 문제를 해소했으며, 문장의 첫 단어로부터 해당 단어 직전의 단어까지 모두 조건부애 넣어 확률을 근사할 수 있습니다.

<br></br>
$$ P(w_1, w_2, \dots, w_k) = \prod_{i=1}^k P(w_i|w_{<i}) $$
<br></br>

로그를 취하여 합으로 표현하면 다음과 같습니다.

<br></br>
$$ log P(w_1, w_2, \dots, w_k) = \sum_{i=1}^k log P(w_i|w_{<i}) $$
<br></br>

### 구현

RNNLM을 수식화하면 다음과 같습니다.

<br></br>
$$
x_{1:n} = \{x_0, x_1, \dots, x_n, x_{n+1} \} \\
\text{where}\ x_0 = BOS, x_{n+1} = EOS \\
\\
\hat{x_{i+1}} =  softmax(linear_{\text{hidden_size} \to |V|}(RNN(emb(x_i)))) \\
\hat{x_{1:n}}[1:] =  softmax(linear_{\text{hidden_size} \to |V|}(RNN(emb(x_{1:n}[:-1])))) \\
\\
linear_{d_1 \to d_2}(x) = Wx + b\ \text{where}\ W \in R^{d_1 x d_2}, b \in R^d
$$
<br></br>

이때 입력 문장의 시작과 끝에는 x_0와 x_{n+1}이 추가되어 BOS와 EOS를 나타냅니다. 따라서 실제 문장을 나타내는 시퀀스의 길이는 2만큼 더 늘어납니다.

수식을 과정별로 따라가보면, 먼저 문자 $x_{1:n}[:-1]$을 입력으로 받아 각 time-step별 토큰 $x_i$로 임베딩 계층 emb에 넣어 정해진 차원의 단어 임베딩 벡터를 얻습니다. 여기서 주의할 점은 EOS를 떼고 임베딩 계층에 입력으로 주어진다는 것입니다.

<br></br>
$$
x_{1:n}[:-1] = \{x_0, x_1, \dots, x_n \} \\
x_{emb} = emb(x_{1:n}[:-1]) \\
\\
\text{where}\ |x_{1:n}[:-1]| = \text{(batch_size, n+1)} \\
|x_{emb}| = \text{(batch_size, n+1, word_vec_dim)}
$$
<br></br>

RNN은 해당 단어 임베딩 벡터를 입력으로 받고, RNN의 은닉 상태의 크기인 hidden_size의 벡터를 반환합니다. 이 과정은 파이토치를 통해 문장의 모든 time-step을 한번에 병렬로 계산할 수 있습니다.

<br></br>
$$
h_{0:n} = RNN(x_{emb}) \\
\text{where}\ |h_{0:n}| = \text{(batch_size, n+1, hidden_size)}
$$
<br></br>

이 텐서에 선형 계층과 softmax 함수를 적용하여 각 단어에 대한 확률 분포인 $\hat{x_{i+1}}$를 구합니다.

<br></br>
$$
\hat{x_{1:n+1}} = softmax(linear_{\text{hidden_size} \to |V|}(h_{0:n})) \\
\text{where}\ |\hat{x_{1:n+1}}| = \text{(batch_size, n+1, |V|)} \\
x_{1:n+1}[1:] = \{x_1, x_2, \dots, x_{n+1} \}
$$
<br></br>

여기서 우리는 LSTM을 사용하여 RNN을 대체합니다. LSTM은 여러 계층으로 구성되며, 각 계층 사이에는 드롭아웃이 들어갈 수 있습니다. 테스트 데이터셋에 대한 PPL을 최소화하는 것이 목표인 만큼, 교차 엔트로피 손실 함수를 사용하여 최적화를 수행합니다.

이때 주의할 점은 입력과 반대로 BOS를 제거한 후 비교해야한다는 것입니다.

<br></br>
$$
L(\hat{x_{1:n+1}}, x_{1:n+1}[1:]) = - \frac{1}{n} \sum_{j=1}^{n+1}x_j log \hat{x_j}
$$
<br></br>

### 파이토치 예제

[language_model.py](https:/github.com/kh-kim/OpenNLMTK)

```python
import torch
import torch.nn as nn

import data_loader

class LanguageModel(nn.Module):
    
    def __init__(self,
                 vocab_size,
                 word_vec_dim = 512,
                 hidden_size = 512,
                 n_layers = 4,
                 dropout_p = 0.2,
                 max_length = 255):
        
        self.vocab_size = vocab_size
        self.word_vec_dim = word_vec_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        super(LanguageModel, self).__init__()
        
        self.emb = nn.Embedding(vocab_size,
                                word_vec_dim,
                                padding_idx = data_loader.PAD)
        
        self.rnn = nn.LSTM(word_vec_dim,
                           hidden_size,
                           n_layers,
                           batch_first = True,
                           dropout = dropout_p)
        
        self.out = nn.Linear(hidden_size, vocab_size, bias = True)
        self.log_softmax = nn.LogSoftmax(dim = 2)
        
    def forward(self, x):
        ## |x| = (batch_size, length)
        
        ## |x| = (batch_size, length, word_vec_dim)
        x = self.emb(x)
        
        ## |x| = (batch_size, length, hidden_size)
        x, (h, c) = self.rnn(x)
        
        ## |x| = (batch_size, length, vocab_size)
        x = self.out(x)

        y_hat = self.log_softmax(x)
        
        return y_hat
    
    def search(self, batch_size = 64, max_length = 255):
        ## |x| = (batch_size, 1)        
        x = torch.LongTensor(batch_size, 1).to(next(self.parameters()).device).zero_() + data_loader.BOS
        
        is_undone = x.new_ones(batch_size, 1).float()
        
        y_hats, indice = [], []
        h, c, = None, None
        
        while is_undone.sum() > 0 and len(indice) < max_length:
            ## |emb_t| = (batch_size, 1, word_vec_dim)
            x = self.emb(x)
            
            ## |x| = (batch_size, 1, hidden_size)
            x, (h, c) = self.rnn(x, (h, c)) if h is not None and c is not None else self.rnn(x)
            
            ## |y_hat| = (batch_size, 1, output_size)
            y_hat = self.log_softmax(x)
            
            y_hats += [y_hat]
            
            y = torch.multinomial(y_hat.exp().view(batch_size, -1), 1)
            
            ## |y| = (batch_size, 1)            
            y = y.masked_fill_((1. - is_undone).byte(), data_loader.PAD)
            
            ## |is_undone| = (batch_size, 1)
            is_undone = is_undone * torch.ne(y, data_loader.EOS).float()
            
            indice += [y]
            
            x = y
        
        ## |y_hat| = (batch_size, length, output_size)
        y_hats = torch.cat(y_hats, dim = 1)
        
        ## |indice| = (batch_size, length)
        indice = torch.cat(indice, dim = -1)
        
        return y_hats, indice
```

NNLM은 단어 임베딩 벡터를 사용하여 희소성 문제를 해결하는데 큰 효과를 보았습니다. 따라서 훈련 데이터셋에서 보지 못한 단어의 조합에 대해서도 훌륭하게 대처할 수 있습니다.